## Steps
1. read data
2. time differential
3. PSD (using welch)
4. extract features
5. match labels (target)
6. add patient id

In [1]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
from scipy.stats import entropy
from sampen import sampen2


In [2]:
# subject_id = 23
# base_path = "physionet.org/files/chbmit/1.0.0/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "chb{:02d}/*.edf".format(subject_id))))
# summary_file = os.path.join(base_path, "chb{:02d}/chb{:02d}-summary.txt".format(subject_id, subject_id))


# summary_content = open(summary_file,'r').read()

In [2]:
def extract_data_and_labels(edf_filename, summary_text):
    folder, basename = os.path.split(edf_filename)
    
    edf = mne.io.read_raw_edf(edf_filename,stim_channel=None)
    # X = edf.get_data().astype(np.float32) * 1e6 # to mV
    X= shuffle_channels(edf).astype(np.float32)
    print (X.shape)
    y = np.zeros(X.shape[1], dtype=np.int64)
    i_text_start = summary_text.index(basename)

    if 'File Name' in summary_text[i_text_start:]:
        i_text_stop = summary_text.index('File Name', i_text_start)
    else:
        i_text_stop = len(summary_text)
    assert i_text_stop > i_text_start

    file_text = summary_text[i_text_start:i_text_stop]
    # print(file_text)
    num_seizures = int(re.search(r"Number of Seizures in File: ([0-9]*)", file_text).group(1))
    
    # if 'Seizure ' in file_text:
    if num_seizures>0:
        for sez in range(1, num_seizures+1):
            # print(re.search(rf"Seizure[\s]*[{str(sez)}]* Start Time:[\s]* ([0-9]*) seconds", file_text))
            start_sec = int(re.search(rf"Seizure[\s]*[{str(sez)}]* Start Time:[\s]* ([0-9]*) seconds", file_text).group(1))
            print(start_sec)
            end_sec = int(re.search(rf"Seizure[\s]*[{str(sez)}]* End Time:[\s]* ([0-9]*) seconds", file_text).group(1))
            print(end_sec)
            i_seizure_start = int(round(start_sec * edf.info['sfreq']))
            i_seizure_stop = int(round((end_sec + 1) * edf.info['sfreq']))
            y[i_seizure_start:i_seizure_stop] = 1
    assert X.shape[1] == len(y)
    return X,y

In [3]:
channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
def shuffle_channels(data):
    channels=data.ch_names
    d=data.get_data().astype(np.float32) * 1e6 
    res= [0]*23
    # res=np.zeros_like(res)
    i=0
    for ch in channels:
        if channel_order.get(ch)!=None:
            idx=channel_order[ch]
            res[idx]=np.array(d[i])
        i+=1
    r=np.zeros_like(res)
    for i in range(0,23):
        r[i]=res[i]
    return r

In [5]:
# X_23 = []
# y_23 = []
# for edf_file_name in edf_file_names:
#     X, y = extract_data_and_labels(edf_file_name, summary_content)
#     X_23.append(X)
#     y_23.append(y)

In [6]:
# len(X_23)

In [4]:
def time_differential(x_series):
    a=[]
    for ch in x_series:
        c=np.zeros_like(ch)
        for i in range (1,len(ch)):
            c[i]=ch[i]-ch[i-1]
        a.append(c)
    return np.array(a)

    

In [5]:
label_dict ={0: 'interictal', 1: 'ictal', 2: 'preictal', 3: 'dismiss'}

In [6]:
def label_series(labels, preictalmins):
    interictal=60*60*256
    preictal=preictalmins*60*256
    # interictal=4
    # preictal=2
    seizure_starts=[]
    seizure_ends=[]
    if labels[0]==1:
        seizure_starts.append(0)
    for i in range(1,len(labels)):
        if labels[i]==1 and labels[i-1]==0:
            seizure_starts.append(i)
        if labels[i]==0 and labels[i-1]==1:
            seizure_ends.append(i)
    # print(seizure_starts)
    # print(seizure_ends)
    for point in seizure_starts:
        # print(point)
        for j in range(max(0, point-preictal), point):
            if labels[j]!=1:
                labels[j]=2
        for k in range(max(0, point-interictal), point):
            if labels[k]==0:
                labels[k]=3
    for point in seizure_ends:
        for j in range(point, min(interictal+ point, len(labels))):
            if labels[j]==0:
                labels[j]=3
    return labels
        

           



In [7]:
def check_channel_match(channels):
    i=0
    for ch in channels:
        if channel_order.get(ch)==None or channel_order[ch]!=i:
            return False
        i+=1
    return True


In [8]:
def extract_frequency(freqs, psd, start, end):
    start_idx=np.where(np.isclose(freqs, start))[0][0]
    if end==-1:
        end_idx=-1
    else:
        end_idx=np.where(np.isclose(freqs, end))[0][0]
    return psd[start_idx:end_idx]


In [9]:
def set_frequency(freqs, psd, start, end, value):
    start_idx=np.where(np.isclose(freqs, start))[0][0]
    if end==-1:
        end_idx=-1
    else:
        end_idx=np.where(np.isclose(freqs, end))[0][0]    
    psd[start_idx:end_idx]=value

In [10]:
from scipy.stats.stats import pearsonr

def order_correlation(data, target):
    df=pd.DataFrame(data=data, columns=generate_column_names())
    df['target']=target
    df=df.dropna()
    df=df[(df['target']!=3)]
    coefs=np.zeros(len(df.columns)-2)
    p_vals=np.zeros(len(df.columns)-2)
    for i in range(0,len(df.columns)-2):
        (coefs[i], p_vals[i])=pearsonr(df[df.columns[i]], df['target'])
    corr_df=pd.DataFrame(data={'corr01':coefs,'p01': p_vals})
    corr_df['feature']=df.columns[:-2]
    corr_df=corr_df.sort_values('corr01', ascending=False)
    return np.append(corr_df['feature'], 'patient')

In [11]:
def generate_column_names():
    channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
    col_names=[]
    for ch in channel_order:
        # print(ch)
        for i in range(0,9):
            col_names.append(str(ch)+'-'+str(i))
    col_names.append('patient')
    return col_names

In [15]:
# patient_no=1#################

# for patient_no in range(1,6):
#     subject_id = patient_no
#     base_path = "physionet.org/files/chbmit/1.0.0/"
#     edf_file_names = sorted(glob.glob(os.path.join(base_path, "chb{:02d}/*.edf".format(subject_id))))
#     summary_file = os.path.join(base_path, "chb{:02d}/chb{:02d}-summary.txt".format(subject_id, subject_id))


#     summary_content = open(summary_file,'r').read()

#     X_all = []
#     y_all = []
#     for edf_file_name in edf_file_names:
#         X, y = extract_data_and_labels(edf_file_name, summary_content)
#         X_all.append(X)
#         y_all.append(y)

#     window_size=int(256*20)
#     for filno in range(0,len(X_all)):###################
#         recording=X_all[filno]####################
#         discarded=0
#         wins=recording.shape[1]//window_size
#         print('file: {} , shape: {} '.format(filno,recording.shape))
#         y=label_series(y_all[filno], 10)#############################33333
#         datapoints_in=[] #for each file
#         targets=[]
#         i=0
#         while i+window_size<recording.shape[1]:
#             # n_channels=recording.shape[0]
#             n_channels=23
#             # features=np.zeros(n_channels*9+1)
#             features=np.zeros(3)
#             features[2]=patient_no
#             recording.shape
#             window=time_differential(recording[:,int(i):int(i+window_size)])
#             # window=recording[:,int(i):int(i+window_size)]
#             freqs, psd = signal.periodogram(window, fs=256)
#             features[0]=psd.sum()
#             features[1]=np.sqrt(np.mean(psd**2))
#             target_range=np.array(y[int(i):int(i+window_size)])

#             if np.all(target_range == target_range[0]):
#                 target=target_range[0]
#                 targets.append(target)
#                 datapoints_in.append(features)
#             else:
#                 discarded+=1               
#             i+=(window_size/2)
#         # dp=np.array(datapoints_in)
#         # D:\salma\UNI\Sem8\WS\unnormalized
#         save(os.path.join('sum_only','data_chb{:02d}/features_{}_{:02d}'.format(patient_no, patient_no, filno)), datapoints_in)
#         save(os.path.join('sum_only','data_chb{:02d}/targets_{}_{:02d}'.format(patient_no,patient_no, filno)) , targets)
#         print('total number of windows: {} , number of windows discarded: {} '.format(i/(window_size/2), discarded))
        



- Channel 1: FP1-F7  ----0
- Channel 2: F7-T7   ----1
- Channel 3: T7-P7   ----2
- Channel 4: P7-O1   ----3
- Channel 5: -
- Channel 6: FP1-F3  ----4
- Channel 7: F3-C3   ----5
- Channel 8: C3-P3   ----6
- Channel 9: P3-O1   ----7
- Channel 10: -
- Channel 11: FZ-CZ   ----16
- Channel 12: CZ-PZ   ----17
- Channel 13: -
- Channel 14: FP2-F4   ----8
- Channel 15: F4-C4   ----9
- Channel 16: C4-P4  ----10
- Channel 17: P4-O2   ----11
- Channel 18: -
- Channel 19: FP2-F8   ----12
- Channel 20: F8-T8   ----13
- Channel 21: T8-P8   ----14
- Channel 22: P8-O2   ----15
- Channel 23: -
- Channel 24: P7-T7   ----18
- Channel 25: T7-FT9   ----19
- Channel 26: FT9-FT10   ----20
- Channel 27: FT10-T8   ----21
- Channel 28: T8-P8   ----22


In [14]:
patient_no=1#################

for patient_no in range(1,20):
    subject_id = patient_no
    base_path = "physionet.org/files/chbmit/1.0.0/"
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "chb{:02d}/*.edf".format(subject_id))))
    summary_file = os.path.join(base_path, "chb{:02d}/chb{:02d}-summary.txt".format(subject_id, subject_id))


    summary_content = open(summary_file,'r').read()

    X_all = []
    y_all = []
    for edf_file_name in edf_file_names:
        X, y = extract_data_and_labels(edf_file_name, summary_content)
        X_all.append(X)
        y_all.append(y)

    window_size=int(256*20)
    for filno in range(0,len(X_all)):###################
        recording=X_all[filno]####################
        discarded=0
        wins=recording.shape[1]//window_size
        print('file: {} , shape: {} '.format(filno,recording.shape))
        y=label_series(y_all[filno], 10)#############################33333
        datapoints_in=[] #for each file
        targets=[]
        i=0
        while i+window_size<recording.shape[1]:
            # n_channels=recording.shape[0]
            n_channels=23
            features=np.zeros(25)
            # features[-1]=patient_no
            recording.shape
            # window=time_differential(recording[:,int(i):int(i+window_size)])
            window=recording[:,int(i):int(i+window_size)]
            freqs, psd = signal.periodogram(window, fs=256)
            #removing electric noise
            # psd[:,57:64]=0
            # psd[:,117:124]=0
            target_range=np.array(y[int(i):int(i+window_size)])
            for chi in range (0, n_channels):
                set_frequency(freqs, psd[chi],57,64, 0)
                set_frequency(freqs, psd[chi],117,124, 0)
            features[0]=np.sqrt(np.mean(extract_frequency(freqs, psd[1],0.5, 31)**2))
            # print(sampen2(extract_frequency(freqs, psd[1],0.5, 5))[2][1])
            try:
                features[1]=sampen2(extract_frequency(freqs, psd[1],0.5, 5))[-1][1]
            except ZeroDivisionError:
                features[1]=float('NaN')
            features[2]=np.sqrt(np.mean(extract_frequency(freqs, psd[1],4, 9)**2))
            features[3]=np.sqrt(np.mean(extract_frequency(freqs, psd[1],0.5, 5)**2))
            try:
                features[4]=sampen2(extract_frequency(freqs, psd[0],0.5,5 ))[-1][1]
            except ZeroDivisionError:
                features[4]=float('NaN')   
            features[5]=np.sqrt(np.mean(extract_frequency(freqs, psd[15],0.5, 31)**2))
            features[6]=np.sqrt(np.mean(extract_frequency(freqs, psd[15],0.5, 5)**2))
            features[7]=np.sqrt(np.mean(extract_frequency(freqs, psd[11],12, 31)**2))
            features[8]=np.sqrt(np.mean(extract_frequency(freqs, psd[15],12, 31)**2))
            features[9]=np.sqrt(np.mean(extract_frequency(freqs, psd[15],4, 9)**2))
            features[10]=np.sqrt(np.mean(extract_frequency(freqs, psd[2],0.5, 31)**2))
            features[11]=np.sqrt(np.mean(extract_frequency(freqs, psd[2],0.5, 5)**2))
            features[12]=np.sqrt(np.mean(extract_frequency(freqs, psd[3],4, 9)**2))
            features[13]=np.median(extract_frequency(freqs, psd[2],0.5, 5))
            features[14]=np.sqrt(np.mean(extract_frequency(freqs, psd[3],0.5, 31)**2))
            features[15]=np.sqrt(np.mean(extract_frequency(freqs, psd[17],4, 9)**2))
            features[16]=np.sqrt(np.mean(extract_frequency(freqs, psd[17],0.5, 31)**2))
            features[17]=np.sqrt(np.mean(extract_frequency(freqs, psd[16],0.5, 31)**2))
            features[18]=np.sqrt(np.mean(extract_frequency(freqs, psd[16],0.5, 5)**2))
            features[19]=np.sqrt(np.mean(extract_frequency(freqs, psd[17],0.5, 5)**2))
            features[20]=np.sqrt(np.mean(extract_frequency(freqs, psd[20],0.5, 31)**2))
            features[21]=np.sqrt(np.mean(extract_frequency(freqs, psd[20],4, 9)**2))
            features[22]=np.sqrt(np.mean(extract_frequency(freqs, psd[20],12, 31)**2))
            features[23]=np.sqrt(np.mean(extract_frequency(freqs, psd[20],8, 13)**2))
            features[24]=np.sqrt(np.mean(extract_frequency(freqs, psd[20],0.5, 5)**2))



                # # alpha=np.sum(psd[chi,8:14])
                # alpha =extract_frequency(freqs, psd[chi],8,14)
                # # beta=np.sum(psd[chi,13:31])
                # beta = extract_frequency(freqs, psd[chi],13,31)
                # # delta=np.sum(psd[chi,1:5])
                # delta =extract_frequency(freqs, psd[chi],0.5,5)
                # # theta=np.sum(psd[chi, 4:9])
                # theta =extract_frequency(freqs, psd[chi],4,9)
                # # gamma0=np.sum(psd[chi,30:48])
                # gamma0=extract_frequency(freqs, psd[chi],30,48)
                # # gamma1=np.sum(psd[chi,53:76])
                # gamma1=extract_frequency(freqs, psd[chi],53,76)
                # # gamma2=np.sum(psd[chi,75:98])
                # gamma2=extract_frequency(freqs, psd[chi],75,98)
                # # gamma3=np.sum(psd[chi,103:])
                # gamma3=extract_frequency(freqs, psd[chi],103,-1)
                # features[chi*9:(1+chi)*9-1]=[alpha,beta,delta,theta,gamma0,gamma1,gamma2,gamma3]
                # # total_power=np.sum(features[chi*9:(1+chi)*9-1])
                # total_power=np.sum(psd[chi])
                # features[chi*9:(1+chi)*9-1] /= total_power
                # features[(1+chi)*9-1]=total_power
            if np.all(target_range == target_range[0]):
                target=target_range[0]
                targets.append(target)
                datapoints_in.append(features)
            else:
                discarded+=1               
            i+=(window_size/2)
        # dp=np.array(datapoints_in)
        # D:\salma\UNI\Sem8\WS\unnormalized
        save(os.path.join('top25_nondiff','data_chb{:02d}/features_{}_{:02d}'.format(patient_no, patient_no, filno)), datapoints_in)
        save(os.path.join('top25_nondiff','data_chb{:02d}/targets_{}_{:02d}'.format(patient_no,patient_no, filno)) , targets)
        print('total number of windows: {} , number of windows discarded: {} '.format(i/(window_size/2), discarded))
        



following channels:
--0, --1, --2, --3, --4
  edf = mne.io.read_raw_edf(edf_filename,stim_channel=None)
(23, 921600)
Extracting EDF parameters from d:\salma\UNI\Sem8\WS\physionet.org\files\chbmit\1.0.0\chb19\chb19_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
<ipython-input-2-0a23d589a01a>:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf = mne.io.read_raw_edf(edf_filename,stim_channel=None)
<ipython-input-2-0a23d589a01a>:4: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  edf = mne.io.read_raw_edf(edf_filename,stim_channel=None)
(23, 921600)
Extracting EDF parameters from d:\salma\UNI\Sem8\WS\physionet.org\files\chbmit\1.0.0\chb19\chb19_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
<ipython-input-2-0a23d589a01a>:4: RuntimeWarning: Channel names are not unique, foun

In [17]:

data = load('features_17_12.npy')
# data2 = load('features/data_chb01/targets_1_0.npy')
len(data)

FileNotFoundError: [Errno 2] No such file or directory: 'features_17_12.npy'

In [63]:
save('features_{}_{}.npy'.format(17, 12), data)


In [65]:
data.shape

(358,)

In [61]:
data=data[0:147]

In [32]:
np.isnan(data).argmax()

30576

In [33]:
30576

147.0

In [152]:
y_06[0][13525*256:13526*256]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [53]:
50344/208

242.03846153846155

In [2]:
256*60


15360

In [91]:
y_01[3]
np.savetxt("array_3d.csv",[y_01[3]],delimiter='\n',fmt="%d")


In [ ]:
edf_filename= 'physionet.org/files/chbmit/1.0.0/chb12/chb12_32.edf'
edf = mne.io.read_raw_edf(edf_filename,stim_channel=None)
X = edf.get_data().astype(np.float32) * 1e6 # to mV
print(edf.get_data()[10])
check_channel_match(edf.ch_names)
print(shuffle_channels(edf)[10])
